In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform for dataset - Normalizing with mean and std of Fashion MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Fashion MNIST is grayscale
])

# Download and load the training and test data
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [3]:
class FashionMNISTClassifier(nn.Module):
    def __init__(self):
        super(FashionMNISTClassifier, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)  # Input layer (flattened 28x28 images)
        self.fc2 = nn.Linear(256, 128)      # Hidden layer
        self.fc3 = nn.Linear(128, 10)       # Output layer (10 classes for Fashion MNIST)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)          # No activation on the output layer
        return x

# Instantiate the model, define loss function and optimizer
model = FashionMNISTClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [4]:
# Function for training the model
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / len(train_loader)

# Function for testing the model
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            # Calculate the batch loss
            test_loss += criterion(outputs, labels).item()
            
            # Get the predictions
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss / len(test_loader), accuracy

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, accuracy = test(model, test_loader, criterion, device)
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch 1/10, Train Loss: 0.4918, Test Loss: 0.4278, Accuracy: 84.39%
Epoch 2/10, Train Loss: 0.3710, Test Loss: 0.3806, Accuracy: 86.07%
Epoch 3/10, Train Loss: 0.3311, Test Loss: 0.3622, Accuracy: 87.17%
Epoch 4/10, Train Loss: 0.3030, Test Loss: 0.3688, Accuracy: 86.59%
Epoch 5/10, Train Loss: 0.2831, Test Loss: 0.3393, Accuracy: 87.76%
Epoch 6/10, Train Loss: 0.2677, Test Loss: 0.3621, Accuracy: 86.95%
Epoch 7/10, Train Loss: 0.2547, Test Loss: 0.3472, Accuracy: 87.53%
Epoch 8/10, Train Loss: 0.2411, Test Loss: 0.3390, Accuracy: 88.43%
Epoch 9/10, Train Loss: 0.2284, Test Loss: 0.3390, Accuracy: 88.34%
Epoch 10/10, Train Loss: 0.2199, Test Loss: 0.3484, Accuracy: 88.49%


In [5]:
torch.save(model.state_dict(), 'fashion_mnist_ann.pth')